In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os, pickle, utils

from tqdm.notebook import tqdm
from datetime import timedelta
#from utils import baseline_SCr

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu  = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'
ed   = './data/mimic-iv-2.2-parquet/ed/'

pd.set_option('mode.chained_assignment',  None)

In [12]:
hosp_list = ['labevents', 'd_labitems', 'patients', 'admissions', 
             'diagnoses_icd', 'microbiologyevents', 'prescriptions', 'omr']
for i in hosp_list:
    globals()['{}'.format(i)] = pd.read_parquet(hosp+i+'.parquet')

In [13]:
icu_list = ['chartevents', 'd_items', 'icustays', 'inputevents', 'outputevents', 'procedureevents']
for i in icu_list:
    globals()['{}'.format(i)] = pd.read_parquet(icu+i+'.parquet')

In [14]:
vitalsign = pd.read_parquet(ed+'vitalsign.parquet')

In [15]:
labvalues = pd.read_csv('./data/origin/labvalues/labvalues.csv')
vitals = pd.read_csv('./data/origin/vitals/vitals.csv')
comorbidities = pd.read_csv('./data/origin/demographic/comorbidities.csv')

# Demographic

## Gender

In [16]:
if not os.path.isfile('./data/origin/demographic/patients_gender.parquet'):
    patients_gender = utils.cal_gender(patients)
    patients_gender.to_parquet('./data/origin/demographic/patients_gender.parquet')
else : patients_gender = pd.read_parquet('./data/origin/demographic/patients_gender.parquet')

## Age

In [17]:
if not os.path.isfile('./data/origin/demographic/icustays_age.parquet'):
    icustays_age = utils.cal_age(icustays,patients)
    icustays_age.to_parquet('./data/origin/demographic/icustays_age.parquet')
else : icustays_age = pd.read_parquet('./data/origin/demographic/icustays_age.parquet')

## Race

In [18]:
if not os.path.isfile('./data/origin/demographic/admissions_race.parquet'):
    admissions_race = utils.cal_race(admissions)
    admissions_race.to_parquet('./data/origin/demographic/admissions_race.parquet')
else : admissions_race = pd.read_parquet('./data/origin/demographic/admissions_race.parquet')

## Height

In [ ]:
#admission_height = cal_height(icustays,chartevents,omr)

## Weight

In [19]:
if not os.path.isfile('./data/origin/demographic/admission_weight.parquet'):
    admission_weight = utils.cal_weight(icustays,chartevents,inputevents,omr)
    admission_weight.to_parquet('./data/origin/demographic/admission_weight.parquet')
else : admission_weight = pd.read_parquet('./data/origin/demographic/admission_weight.parquet')

## Comorbidites

In [20]:
if not os.path.isfile('./data/origin/demographic/comorbidities.parquet'):
    comorbidities = utils.cal_comorbidities(icustays, comorbidities,diagnoses_icd)
    comorbidities.to_parquet('./data/origin/demographic/comorbidities.parquet')
else : comorbidities = pd.read_parquet('./data/origin/demographic/comorbidities.parquet')

## Baseline SCr

In [21]:
if not os.path.isfile('./data/origin/demographic/baseline_SCr.parquet'):
    baseline_SCr = utils.cal_baseline_SCr(labevents,icustays,patients_gender,icustays_age,admissions_race)
    baseline_SCr.to_parquet('./data/origin/baseline_SCr.parquet')
else : baseline_SCr = pd.read_parquet('./data/origin/demographic/baseline_SCr.parquet')

# Lab values

In [ ]:
for i,idx in enumerate(tqdm(labvalues.abbreviation)):
    if not os.path.isfile('./data/origin/labvalues/chartevents_%s.parquet'%idx):
        tmp = labvalues.iloc[i]
        globals()['chartevents_{}'.format(idx)] = utils.extract_labvalues(chartevents,labevents,tmp,is_in_icu=True)
        globals()['chartevents_{}'.format(idx)].to_parquet('./data/origin/labvalues/chartevents_%s.parquet'%idx)
    else : globals()['chartevents_{}'.format(idx)] = pd.read_parquet('./data/origin/labvalues/chartevents_%s.parquet'%idx)

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
for i,idx in enumerate(tqdm(labvalues.abbreviation)):
    if not os.path.isfile('./data/origin/labvalues/labevents_%s.parquet'%idx):
        tmp = labvalues.iloc[i]
        globals()['labevents_{}'.format(idx)] = utils.extract_labvalues(chartevents,labevents,tmp,is_in_icu=False)
        globals()['labevents_{}'.format(idx)].to_parquet('./data/origin/labvalues/labevents_%s.parquet'%idx)
    else : globals()['labevents_{}'.format(idx)] = pd.read_parquet('./data/origin/labvalues/labevents_%s.parquet'%idx)

  0%|          | 0/30 [00:00<?, ?it/s]

In [22]:
for i,idx in enumerate(tqdm(labvalues.abbreviation)):
    if not os.path.isfile('./data/resample/labvalues/resample_%s.parquet'%idx):
        globals()['resample_{}'.format(idx)] = utils.resample_labvalues(globals()['chartevents_{}'.format(idx)],globals()['labevents_{}'.format(idx)],icustays,idx)
        globals()['resample_{}'.format(idx)].to_parquet('./data/resample/labvalues/resample_%s.parquet'%idx)
    else : globals()['resample_{}'.format(idx)] = pd.read_parquet('./data/resample/labvalues/resample_%s.parquet'%idx)

  0%|          | 0/30 [00:00<?, ?it/s]

# Vitals

In [ ]:
for i,idx in enumerate(tqdm(vitals.abbreviation)):
    if not os.path.isfile('./data/origin/vitals/chartevents_%s.parquet'%idx):
        tmp = vitals.iloc[i]
        globals()['chartevents_{}'.format(idx)] = utils.extract_labvalues(chartevents,labevents,tmp,is_in_icu=True)
        globals()['chartevents_{}'.format(idx)].to_parquet('./data/origin/vitals/chartevents_%s.parquet'%idx)
    else : globals()['chartevents_{}'.format(idx)] = pd.read_parquet('./data/origin/vitals/chartevents_%s.parquet'%idx)

# Temperature
chartevents_tempF['valuenum'] = (chartevents_tempF['valuenum']-32)*5/9
chartevents_tempF.rename(columns={'valuenum2':'valuenum'},inplace=True)
chartevents_tempF['valuenum'] = chartevents_tempF['valuenum'].round(1)

chartevents_temp = pd.concat([chartevents_tempC,chartevents_tempF])
chartevents_temp.sort_values(by=['subject_id','hadm_id','stay_id','charttime'],inplace=True)
chartevents_temp.to_parquet('./data/origin/vitals/chartevents_temp.parquet')

os.remove('./data/origin/vitals/chartevents_tempC.parquet')
os.remove('./data/origin/vitals/chartevents_tempF.parquet')

  0%|          | 0/11 [00:00<?, ?it/s]

In [23]:
for idx in tqdm([x[12:-8] for x in os.listdir('./data/origin/vitals') if x[-8:] == '.parquet']):
    print('Processing %s'%idx)
    if not os.path.isfile('./data/resample/vitals/resample_%s.parquet'%idx):
        globals()['resample_{}'.format(idx)] = utils.resample_vitals(globals()['chartevents_{}'.format(idx)],icustays,idx)
        globals()['resample_{}'.format(idx)].to_parquet('./data/resample/vitals/resample_%s.parquet'%idx)
    else : globals()['resample_{}'.format(idx)] = pd.read_parquet('./data/resample/vitals/resample_%s.parquet'%idx)

  0%|          | 0/10 [00:00<?, ?it/s]

Processing ABPd
Processing ABPs
Processing CVP
Processing FiO2
Processing HR
Processing NBPd
Processing NBPs
Processing RR
Processing SaO2
Processing temp


In [24]:
resample_SBP = utils.cal_BP(resample_ABPs, resample_NBPs, isSBP=True)
resample_DBP = utils.cal_BP(resample_ABPd, resample_NBPd, isSBP=False)
resample_MAP = utils.cal_MAP(resample_DBP, resample_SBP)

resample_SBP.to_parquet('./data/resample/vitals/resample_SBP.parquet')
resample_DBP.to_parquet('./data/resample/vitals/resample_DBP.parquet')
resample_MAP.to_parquet('./data/resample/vitals/resample_MAP.parquet')

# Urine Output

In [25]:
if not os.path.isfile('./data/resample/measures/resample_uo.parquet'):
    outputevents_uo = utils.cal_uo(outputevents)
    resample_uo = utils.resample_urine(outputevents_uo, icustays, 'uo')
    resample_uo.to_parquet('./data/resample/measures/resample_uo.parquet')
else : resample_uo = pd.read_parquet('./data/resample/measures/resample_uo.parquet')

# Fluid/Vasopressor intake

## Fluid

In [26]:
if not os.path.isfile('./data/resample/inputs/resample_fluid.parquet'):
    fluid = [220949, 220950, 220952, 225158, 225159, 225161, 225828, 225797, 225799, 225823, 225825, 225827, 225830, 226089, 225941, 225943, 225944, 226361, 226363, 226364, 226375, 226377, 226452, 226453, 227533, 228140, 228141, 228142, 228341, 220955, 220967, 220968, 220953]
    inputevents_fluid = inputevents[inputevents['itemid'].isin(fluid)]

    inputevents_fluid.loc[inputevents_fluid['rateuom']=='mL/min', 'rate'] = inputevents_fluid['rate']/60
    inputevents_fluid.loc[inputevents_fluid['rateuom']=='mL/kg/hour', 'rate'] = inputevents_fluid['rate'] * inputevents_fluid['patientweight']

    resample_fluids = utils.resample_inputrates(icustays,inputevents_fluid,'fluid')
    resample_fluids.to_parquet('./data/resample/inputs/resample_fluid.parquet')
else : resample_fluids = pd.read_parquet('./data/resample/inputs/resample_fluid.parquet')

## Vasopressor

In [27]:
inputevents_angiotensin_ii = inputevents[inputevents['itemid'].isin([229709,229764])]

In [28]:
inputevents_epinephrine = inputevents[inputevents['itemid'].isin([221289])]
inputevents_dopamine = inputevents[inputevents['itemid'].isin([221662])]
inputevents_dobutamine = inputevents[inputevents['itemid'].isin([221653])]
inputevents_norephinephrine = inputevents[inputevents['itemid'].isin([221906])]
inputevents_phenylephrine = inputevents[inputevents['itemid'].isin([221749, 229630, 229632])]
inputevents_vasopressin = inputevents[inputevents['itemid'].isin([222315])]

In [29]:
# Norephinephrine
inputevents_norephinephrine.loc[inputevents_norephinephrine['rateuom']=='mg/kg/min','rate']= inputevents_norephinephrine['rate']*1e3
inputevents_norephinephrine = inputevents_norephinephrine[['subject_id','hadm_id','stay_id','starttime','endtime','rate']]

# Epinephrine
inputevents_epinephrine = inputevents_epinephrine[['subject_id','hadm_id','stay_id','starttime','endtime','rate']]

# Phenylephrine
inputevents_phenylephrine.loc[inputevents_phenylephrine['rateuom']=='mcg/min','rate'] = inputevents_phenylephrine['rate']/inputevents_phenylephrine['patientweight']
inputevents_phenylephrine = inputevents_phenylephrine[['subject_id','hadm_id','stay_id','starttime','endtime','rate']]

# Vasopressin
inputevents_vasopressin.loc[inputevents_vasopressin['rateuom']=='units/hour', 'rate'] = inputevents_vasopressin['rate']/60
inputevents_vasopressin = inputevents_vasopressin[['subject_id','hadm_id','stay_id','starttime','endtime','rate']]

# Dopamine
inputevents_dopamine = inputevents_dopamine[['subject_id','hadm_id','stay_id','starttime','endtime','rate']]

# Dobupamine
inputevents_dobutamine = inputevents_dobutamine[['subject_id','hadm_id','stay_id','starttime','endtime','rate']]

# angiotensin_ii
inputevents_angiotensin_ii.loc[inputevents_angiotensin_ii['rateuom']=='mcg/kg/min', 'rate'] = inputevents_angiotensin_ii['rate']*1e3
inputevents_angiotensin_ii = inputevents_angiotensin_ii[['subject_id','hadm_id','stay_id','starttime','endtime','rate']]

In [30]:
Vaso = ['epinephrine','dopamine','dobutamine','norephinephrine','phenylephrine','vasopressin','angiotensin_ii']
for i,idx in enumerate(tqdm(Vaso)):
    if not os.path.isfile('./data/resample/inputs/resample_%s.parquet'%idx):
        globals()['resample_{}'.format(idx)] = utils.resample_inputrates(icustays, globals()['inputevents_{}'.format(idx)],idx)
        globals()['resample_{}'.format(idx)].to_parquet('./data/resample/inputs/resample_%s.parquet'%idx)
    else : globals()['resample_{}'.format(idx)] = pd.read_parquet('./data/resample/inputs/resample_%s.parquet'%idx)

  0%|          | 0/7 [00:00<?, ?it/s]

In [31]:
for i,idx in enumerate(tqdm(Vaso)):
    if i == 0 : resample_vasopressor = globals()['resample_{}'.format(idx)].copy()
    else : resample_vasopressor = pd.merge(resample_vasopressor,globals()['resample_{}'.format(idx)].copy(),on=['subject_id','hadm_id','stay_id','charttime'],how='left')

  0%|          | 0/7 [00:00<?, ?it/s]

In [32]:
resample_vasopressor['vaso_equ'] = resample_vasopressor['norephinephrine'] + resample_vasopressor['epinephrine'] + 1/100*resample_vasopressor['dopamine'] + 0.06*resample_vasopressor['phenylephrine'] + 2.5*resample_vasopressor['vasopressin'] + 0.0025*resample_vasopressor['angiotensin_ii']
resample_vasopressor = resample_vasopressor[['subject_id','hadm_id','stay_id','charttime','vaso_equ']]

# Ventilator

In [33]:
if not os.path.isfile('./data/resample/procedures/resample_ventilation.parquet'):
    procedureevents_ventilation = procedureevents[procedureevents['itemid'].isin([225792,225794])]
    resample_ventilation = utils.resample_inputrates(icustays,procedureevents_ventilation,'ventilation')
    resample_ventilation.to_parquet('./data/resample_ventilation.parquet')
else : resample_ventilation = pd.read_parquet('./data/resample/procedures/resample_ventilation.parquet')

# RRT

In [34]:
if not os.path.isfile('./data/resample/procedures/resample_rrt.parquet'):
    procedureevents_rrt = procedureevents[procedureevents['itemid'].isin([
    225441, #Hemodialysis
    225802, #Dialysis - CRRT
    225803, #Dialysis - CVVHD
    225805, #Peritoneal Dialysis
    225809, #Dialysis - CVVHDF
    225955, #Dialysis - SCUF
    ])]
    resample_rrt = utils.resample_inputrates(icustays,procedureevents_rrt,'RRT')
    resample_rrt.to_parquet('./data/resample/procedures/resample_rrt.parquet')
else : resample_rrt = pd.read_parquet('./data/resample/procedures/resample_rrt.parquet')

# SOFA

In [35]:
# GCS
chartevents_GCS_eye = chartevents[chartevents['itemid'].isin([220739])]
chartevents_GCS_verval = chartevents[chartevents['itemid'].isin([223900])]
chartevents_GCS_motor = chartevents[chartevents['itemid'].isin([223901])]

GCS = ['GCS_eye','GCS_verval','GCS_motor']
for i,idx in enumerate(tqdm(GCS)):
    if not os.path.isfile('./data/resample/measures/resample_%s.parquet'%idx):
        print("[%i/%i] Resampling %s..."%(i+1,len(vitals.abbreviation.unique()),idx))
        globals()['resample_{}'.format(idx)] = resample_vitals(globals()['chartevents_{}'.format(idx)],icustays,idx)
        globals()['resample_{}'.format(idx)].to_parquet('./data/resample/resample_%s.parquet'%idx)
    else : globals()['resample_{}'.format(idx)] = pd.read_parquet('./data/resample/measures/resample_%s.parquet'%idx)

  0%|          | 0/3 [00:00<?, ?it/s]

In [36]:
resample_GCS_eye = utils.resample_fill(resample_GCS_eye)
resample_GCS_verval = utils.resample_fill(resample_GCS_verval)
resample_GCS_motor = utils.resample_fill(resample_GCS_motor)

resample_GCS_eye.rename(columns={'valuenum':'GCS_eye'},inplace=True)
resample_GCS_verval.rename(columns={'valuenum':'GCS_verval'},inplace=True)
resample_GCS_motor.rename(columns={'valuenum':'GCS_motor'},inplace=True)

resample_GCS_eye.drop('itemid',axis=1,inplace=True)
resample_GCS_verval.drop('itemid',axis=1,inplace=True)
resample_GCS_motor.drop('itemid',axis=1,inplace=True)

resample_GCS = pd.merge(resample_GCS_eye,resample_GCS_verval,on=['subject_id','hadm_id','stay_id','charttime'],how='left')
resample_GCS = pd.merge(resample_GCS,resample_GCS_motor,on=['subject_id','hadm_id','stay_id','charttime'],how='left')

resample_GCS = resample_GCS[['subject_id','hadm_id','stay_id','charttime','GCS_eye','GCS_verval','GCS_motor']]
resample_GCS['GCS'] = resample_GCS['GCS_eye'] + resample_GCS['GCS_verval'] + resample_GCS['GCS_motor']

In [37]:
def resample_PF(resample_PaO2,resample_FiO2):
    resample_PaO2.rename(columns={'valuenum':'PaO2'},inplace=True)
    resample_FiO2.rename(columns={'valuenum':'FiO2'},inplace=True)

    df = pd.merge(resample_PaO2,resample_FiO2,on=['subject_id','hadm_id','stay_id','charttime'],how='left')
    df['PF'] = df['PaO2']/(df['FiO2']/100)
    df = df[['subject_id','hadm_id','stay_id','charttime','PF']]
    return df

resample_PF = resample_PF(resample_PaO2,resample_FiO2)

In [38]:
resample_uo_24hrs = resample_uo.copy()
rolling_avg=resample_uo_24hrs.groupby('stay_id').rolling(window='24H', on='charttime',min_periods=24)['uo'].sum().reset_index(drop=True)
resample_uo_24hrs['uo_day'] = rolling_avg

In [39]:
resample_SOFA = resample_GCS.copy()
for i in ['MAP','dopamine','dobutamine','epinephrine','norephinephrine','PF','Platelet','T_Bil','SCr','ventilation','uo_24hrs']:
    resample_SOFA = pd.merge(resample_SOFA,globals()['resample_{}'.format(i)],on=['subject_id','hadm_id','stay_id','charttime'],how='left')

In [40]:
def cal_SOFA(df):
    # Central nervous system
    #df.loc[df['GCS'].isna(),'SOFA_CNS'] = 0
    df.loc[df['GCS']==15,'SOFA_CNS'] = 0
    df.loc[(df['GCS']>=13)&(df['GCS']<=14),'SOFA_CNS'] = 1
    df.loc[(df['GCS']>=10)&(df['GCS']<=12),'SOFA_CNS'] = 2
    df.loc[(df['GCS']>=6)&(df['GCS']<=9),'SOFA_CNS'] = 3
    df.loc[(df['GCS']<6),'SOFA_CNS'] = 4

    # Cardiovascular system
    #df.loc[df['MAP'].isna(),'SOFA_CVS'] = 0
    df.loc[df['MAP']>=70,'SOFA_CVS'] = 0
    df.loc[df['MAP']<70,'SOFA_CVS'] = 1
    df.loc[((df['dopamine']<=5)&(df['dopamine']>0))|(df['dobutamine']>0),'SOFA_CVS'] = 2
    df.loc[(df['dopamine']>5)|((df['epinephrine']<=0.1)&(df['epinephrine']>0))|((df['norephinephrine']<=0.1)&(df['norephinephrine']>0)),'SOFA_CVS'] = 3
    df.loc[(df['dopamine']>15)|(df['epinephrine']>0.1)|(df['norephinephrine']>0.1),'SOFA_CVS'] = 4

    # Respiratory system
    #df.loc[df['PF'].isna(), 'SOFA_RS'] = 0
    df.loc[df['PF']>=400, 'SOFA_RS'] = 0
    df.loc[df['PF']<400, 'SOFA_RS'] = 1
    df.loc[df['PF']<300, 'SOFA_RS'] = 2
    df.loc[(df['PF']<200)&(df['ventilation']>0), 'SOFA_RS'] = 3
    df.loc[(df['PF']<100)&(df['ventilation']>0), 'SOFA_RS'] = 4

    # Coagulation
    #df.loc[df['Platelet'].isna(), 'SOFA_C'] = 0
    df.loc[df['Platelet']>=150, 'SOFA_C'] = 0
    df.loc[df['Platelet']<150, 'SOFA_C'] = 1
    df.loc[df['Platelet']<100, 'SOFA_C'] = 2
    df.loc[df['Platelet']<50, 'SOFA_C'] = 3
    df.loc[df['Platelet']<20, 'SOFA_C'] = 4

    # Liver
    #df.loc[df['T_Bil'].isna(), 'SOFA_L'] = 0
    df.loc[df['T_Bil']<1.2, 'SOFA_L'] = 0
    df.loc[(df['T_Bil']>=1.2)&(df['T_Bil']<2.0), 'SOFA_L'] = 1
    df.loc[(df['T_Bil']>=2.0)&(df['T_Bil']<6.0), 'SOFA_L'] = 2
    df.loc[(df['T_Bil']>=6.0)&(df['T_Bil']<12.0), 'SOFA_L'] = 3
    df.loc[(df['T_Bil']>=12.0), 'SOFA_L'] = 4

    # Renal function
    #df.loc[df['SCr'].isna(), 'SOFA_R'] = 0
    df.loc[df['SCr']<1.2, 'SOFA_R'] = 0
    df.loc[(df['SCr']>=1.2)&(df['SCr']<2.0), 'SOFA_R'] = 1
    df.loc[(df['SCr']>=2.0)&(df['SCr']<3.5), 'SOFA_R'] = 2
    df.loc[((df['SCr']>=3.5)&(df['SCr']<5.0))|(df['uo_day']<500), 'SOFA_R'] = 3
    df.loc[(df['SCr']>=5.0)|(df['uo_day']<200), 'SOFA_R'] = 4

    df['SOFA'] = df['SOFA_CNS'] + df['SOFA_CVS'] + df['SOFA_RS'] + df['SOFA_C'] + df['SOFA_L'] + df['SOFA_R']
    df = df[['subject_id','hadm_id','stay_id','charttime','SOFA']]
    return df

resample_SOFA = cal_SOFA(resample_SOFA)

In [ ]:
resample_SOFA

,subject_id,hadm_id,stay_id,charttime,SOFA
0,10000032,29079034,39553978,2180-07-23 14:00:00,NaN
1,10000032,29079034,39553978,2180-07-23 15:00:00,NaN
2,10000032,29079034,39553978,2180-07-23 16:00:00,NaN
3,10000032,29079034,39553978,2180-07-23 17:00:00,NaN
4,10000032,29079034,39553978,2180-07-23 18:00:00,NaN
...,...,...,...,...,...
6099429,19999987,23865745,36195440,2145-11-04 16:59:00,6.0
6099430,19999987,23865745,36195440,2145-11-04 17:59:00,6.0
6099431,19999987,23865745,36195440,2145-11-04 18:59:00,6.0
6099432,19999987,23865745,36195440,2145-11-04 19:59:00,6.0


# AKI annotation

In [9]:
resample_AKI_UO = utils.AKI_UO_annotation(resample_uo,admission_weight)

  0%|          | 0/43 [00:00<?, ?it/s]

In [10]:
resample_AKI_UO.AKI_UO.value_counts()

AKI_UO
0    3695199
2     972590
3     967805
1     463840
Name: count, dtype: int64

In [ ]:
resample_AKI_UO.AKI_UO.value_counts()

AKI_UO
0    4441434
2    1017403
1     523221
3     117376
Name: count, dtype: int64

In [41]:
resample_AKI_SCr = utils.AKI_SCr_annotation(resample_SCr,baseline_SCr,resample_rrt)

In [42]:
resample_AKI_SCr.AKI_SCr.value_counts()

AKI_SCr
0    4619467
1     635697
3     588903
2     255367
Name: count, dtype: int64

In [25]:
resample_AKI_SCr.AKI_SCr.value_counts()

AKI_SCr
0    3396458
1    1759748
3     693881
2     249347
Name: count, dtype: int64

# Merge

In [43]:
resample_antibiotics = pd.read_parquet('./data/resample/inputs/resample_antibiotics.parquet')

In [44]:
# merge vital
vitallist = ['HR','SBP','DBP','MAP','temp','RR','CVP','SaO2','FiO2']
for i,idx in enumerate(tqdm(vitallist)):
    if i == 0 : resample_merge = globals()['resample_{}'.format(idx)].copy()
    else : resample_merge = pd.merge(resample_merge,globals()['resample_{}'.format(idx)],on=['subject_id','hadm_id','stay_id','charttime'],how='left')

#merge labvalues
for i,idx in enumerate(tqdm(labvalues.abbreviation)):
    resample_merge = pd.merge(resample_merge,globals()['resample_{}'.format(idx)],on=['subject_id','hadm_id','stay_id','charttime'],how='left')

# merge demographic
resample_merge = pd.merge(resample_merge,patients_gender,on=['subject_id'],how='left')
resample_merge = pd.merge(resample_merge,admissions_race,on=['subject_id'],how='left')
resample_merge = pd.merge(resample_merge,admission_weight,on=['subject_id','stay_id'],how='left')
resample_merge = pd.merge(resample_merge,icustays_age,on=['subject_id','hadm_id','stay_id'],how='left')
resample_merge = pd.merge(resample_merge,comorbidities,on=['subject_id','hadm_id','stay_id'],how='left')
resample_merge = pd.merge(resample_merge,baseline_SCr,on=['subject_id','hadm_id','stay_id'],how='left')

# merge measures
resample_merge = pd.merge(resample_merge,resample_uo,on=['subject_id','hadm_id','stay_id','charttime'],how='left')
resample_merge = pd.merge(resample_merge,resample_SOFA,on=['subject_id','hadm_id','stay_id','charttime'],how='left')
resample_merge = pd.merge(resample_merge,resample_AKI_UO,on=['subject_id','hadm_id','stay_id','charttime'],how='left')
resample_merge = pd.merge(resample_merge,resample_AKI_SCr,on=['subject_id','hadm_id','stay_id','charttime'],how='left')
resample_merge['AKI'] = resample_merge[['AKI_UO','AKI_SCr']].max(axis=1)

# merge procedures & inputs
resample_merge = pd.merge(resample_merge,resample_ventilation,on=['subject_id','hadm_id','stay_id','charttime'],how='left')
resample_merge = pd.merge(resample_merge,resample_fluids,on=['subject_id','hadm_id','stay_id','charttime'],how='left')
resample_merge = pd.merge(resample_merge,resample_vasopressor,on=['subject_id','hadm_id','stay_id','charttime'],how='left')

resample_merge = pd.merge(resample_merge,resample_antibiotics,on=['stay_id','charttime'],how='left')

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [45]:
resample_merge[['Cephalosporins', 'Vancomycin', 'Betalactam_comb', 'Metronidazole',
       'Carbapenems', 'Penicillins', 'Fluoroquinolones', 'Others',
       'count_category']] = resample_merge[['Cephalosporins', 'Vancomycin', 'Betalactam_comb', 'Metronidazole',
       'Carbapenems', 'Penicillins', 'Fluoroquinolones', 'Others',
       'count_category']].fillna(0)

In [ ]:
# Derived variables


In [48]:
resample_merge.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'charttime', 'HR', 'SBP', 'DBP',
       'MAP', 'temp', 'RR', 'CVP', 'SaO2', 'FiO2', 'Alb', 'Alk_Phos', 'AG',
       'BUN', 'Ca', 'CK', 'D_Bil', 'Glu', 'HCT', 'INR', 'PH', 'PHOS',
       'Platelet', 'Cl', 'SCr', 'Na', 'Potassium', 'T_Bil', 'WBC', 'Gl', 'Mg',
       'Ca_ion', 'HCO3', 'AST', 'ALT', 'PTT', 'baseexcess', 'lactate', 'PaO2',
       'PaCO2', 'gender', 'WHITE', 'BLACK', 'HISPANIC OR LATINO', 'ASIAN',
       'MULTIPLE RACE', 'OTHER', 'UNKNOWN', 'valuenum', 'age', 'LD', 'DH',
       'HYP', 'CKD', 'MI', 'DM', 'VD', 'CHF', 'COPD', 'baseline_SCr', 'MDRD',
       'uo', 'SOFA', 'AKI_UO', 'AKI_SCr', 'AKI', 'ventilation', 'fluid',
       'vaso_equ', 'Cephalosporins', 'Vancomycin', 'Betalactam_comb',
       'Metronidazole', 'Carbapenems', 'Penicillins', 'Fluoroquinolones',
       'Others', 'count_category'],
      dtype='object')

In [176]:
resample_merge.count_category.value_counts()

count_category
0.0    5626185
1.0     441531
2.0      30485
3.0       1215
4.0         18
Name: count, dtype: int64

In [47]:
resample_merge.to_parquet('./data/resample/resample_merge.parquet')